### GET LINKS TO ADS FROM SLOVENIA

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
import os
import pandas as pd
import numpy as np
from random import randint
from lxml import etree
import bs4
import datetime


In [16]:
# cookie handler
def cookies():
    try:
        driver.find_element(By.ID,'didomi-notice-agree-button').click()
    except NoSuchElementException:
        pass
    return

In [19]:
# calculate number of pages
def total_pages(page_url):
    driver.get(page_url)
    
    cookies()
    
    # get html of the first page
    page_HTML = bs4.BeautifulSoup(driver.page_source, "lxml")

    # get number of all listed ads
    no_ads = int(page_HTML.find('strong', class_='entities-count').text)
    
    # number of pages = (total number of ads / 25 ads per page) +1
    no_pages = round(((no_ads / 25) +1))
    
    return no_pages

In [14]:
# get html of a page and return it
def get_HTML(url):
    
    sleep_time = 5
    time.sleep(sleep_time)
    
    driver.get(url)
    
    cookies()
    
    html = driver.page_source
    page_HTML = bs4.BeautifulSoup(html, "lxml")

    return page_HTML

In [13]:
# get html page from args and get info about ads on this page (ad ID, link to the ad, time of scraping). Return a df
def scrape_links(page): 
    d = {'Ad_ID':[], 'Ad_Link':[], 'datetime' : []}
    num = page.find('strong', class_="entities-count").text
    soup = page.find('div', class_='EntityList EntityList--Standard EntityList--Regular EntityList--ListItemRegularAd EntityList--itemCount_'+str(num))
    ads = soup.find_all('h3', class_= 'entity-title')
    
    for ad in ads: 
        ad_link = ad.find('a', class_= 'link')['href']
        ad_id = ad_link.split('-')[-1]
        d['Ad_ID'].append(ad_id)
        d['Ad_Link'].append(ad_link)
        d['datetime'].append(datetime.datetime.now().strftime("%d.%m.%Y_%H:%M:%S"))
    d_df = pd.DataFrame(d)
    return d_df

### Main 1

In [24]:
# using chrome browser
PATH = "C:\Program Files (x86)\chromedriver.exe"
data = pd.DataFrame()

# direct link to ads in Slovenia, add page number at the end
url_slo = 'https://www.bolha.com/prodaja-stanovanja?geo%5BlocationIds%5D=26325%2C26324%2C26323%2C26322%2C26321%2C26320%2C26319%2C26318%2C26317%2C26316%2C26315%2C26314&typeOfTransaction=sell&page=' 

# initialize driver
driver = webdriver.Chrome(PATH)

# calculate last page with ads
start_page = 1
end_page = total_pages(url_slo)
print(end_page)


# get info about all listed ads
# end_page+1 because range(a, b) goes from a to b-1 and a=1 in this case (first page)

for i in range(start_page, end_page+1):
    print(i)
    page_url = url_slo + str(i)
    page_html = get_HTML(page_url)
    i_df = scrape_links(page_html)
    data = pd.concat([data, i_df], axis=0, ignore_index=True)

# close driver

# driver.close()

C:\Users\Jernej\AppData\Local\Temp\ipykernel_17660\1972681864.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


43
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43


AttributeError: 'NoneType' object has no attribute 'find_all'

### GET INFO ABOUT INDIVIDUAL AD

In [25]:
# get contents of an ad presented on this page
def scrape_content(page):
    
    # dictionary for info about ad, lists for easier info processing
    ad_dict = {}
    col_list, row_list = [], []

    # adding details about the ad (first published, times viewed...)
    col_ad_info = page.find_all('dt')
    row_ad_info = page.find_all('dd')

    # from all elements get text and strip away unnecessary characters
    for col_item in col_ad_info:
        col_list.append(col_item.text.replace('    ', '').replace('\n','').replace(':',''))
    for row_item in row_ad_info:
        row_list.append(row_item.text.replace('    ', '').replace('\n',''))

    # fill dictionary
    for i in range(0, len(col_list)):
        ad_dict[col_list[i]] = row_list[i]

    return ad_dict


In [29]:
# visit every ad and get info about it, add it to data_ads data frame
def scraping_ads(data):
    # data frame with information about ads' price, location, etc.
    data_ads = pd.DataFrame()

    # base of link to the ad, used in creating a link for a driver to open
    base_url = 'https://www.bolha.com'

    # for every ad get html, scrape info from it and add it to the data frame
    for i in range(0, len(data)):
        print(i) # print current ad number to check progress

        # get html
        urli = base_url+str(data['Ad_Link'][i])
        page = get_HTML(urli)

        # create a dict with info about i-th ad
        i_ad_dict = scrape_content(page)
        i_ad_dict['Link'] = urli
        i_ad_dict['Šifra oglasa'] = data['Ad_ID'][i]

        # create a data frame from dictionary and it append to data_ads 
        ad = pd.DataFrame([i_ad_dict])
     
    	
        data_ads = pd.concat([data_ads, ad], axis=0, ignore_index=True)

    return data_ads

### Main 2

In [32]:
# initialize driver
driver = webdriver.Chrome(PATH) 

# data frame with information about ads' price, location, etc.

data_ads = scraping_ads(data)

# close driver
driver.close()

# data frame to csv
data_ads.to_csv('./bolha_flats_' + str(start_page) + '-' +str(end_page) + '_' + str(datetime.datetime.now().strftime("%d-%m-%Y_%H-%M-%S")) + '.csv')


C:\Users\Jernej\AppData\Local\Temp\ipykernel_17660\2595878652.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


0
        Cena Približno Neto cena DDV Bivalna površina Vrsta stanovanja  \
0  169.900 €                                 66,70 m²          2-sobno   

  Leto izgradnje Vrsta ponudbe                                 Lokacija  \
0           1961        Prodam  Gorenjska, Jesenice, Slovenski Javornik   

      Umeščenost stanovanja  ... Skupno število nadstropij v stavbi  \
0  V večstanovanjski stavbi  ...                                  2   

  Leto zadnje prenove Balkon / terasa / atrij Zunanje površine in oprema  \
0                2020                  Balkon               Vrt/dvorišče   

    Vrsta cene    Oglas je objavljen         Do poteka še Oglas je prikazan  \
0  Skupna cena  14.09.2022. ob 16:17  Št. dni: 29 in 19 h         378 -krat   

                                                Link Šifra oglasa  
0  https://www.bolha.com/nepremicnine/lokacija-st...      9702142  

[1 rows x 21 columns]
1
        Cena Približno Neto cena DDV Bivalna površina Vrsta stanovanja  \
0  147.0